<div style='background-color:#f9f9f9; padding:10px; border-radius:5px; margin-bottom:10px; font-family:Tahoma; color:#000'>
    <h2>Comparison of modeling of Reliance stock prices using different models</h2>
    <h3>By: Puneet Mishra</h3>


### <strong>Combined pipeline</strong>

In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA


class StockExperiment:
    def __init__(
        self,
        tickers,
        start,
        end,
        split_date,
        ma_window=7,
        ar_lags=5,
        arima_order=(5, 0, 0),
        gbm_seed=42
    ):
        self.tickers = tickers
        self.start = start
        self.end = end
        self.split_date = pd.to_datetime(split_date)
        self.ma_window = ma_window
        self.ar_lags = ar_lags
        self.arima_order = arima_order
        self.gbm_seed = gbm_seed

    @staticmethod
    def rmse(y, yhat):
        return np.sqrt(np.mean((y - yhat) ** 2))

    @staticmethod
    def smape(y, yhat):
        return 100 * np.mean(
            2 * np.abs(y - yhat) / (np.abs(y) + np.abs(yhat))
        )

    def gbm(self, train, test):
        np.random.seed(self.gbm_seed)
        log_ret = np.log(train / train.shift(1)).dropna()
        mu = log_ret.mean()
        sigma = log_ret.std()
        S0 = train.iloc[-1]

        dt = 1 / 252
        prices = [S0]
        for _ in range(len(test)):
            z = np.random.normal()
            prices.append(
                prices[-1]
                * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
            )
        return pd.Series(prices[1:], index=test.index)

    def run(self):
        records = []

        for ticker in self.tickers:
            df = yf.download(
                ticker, start=self.start, end=self.end, progress=False, auto_adjust=True
            )

            if df.empty:
                continue

            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(1)

            df = df[["Close"]].dropna()
            df.index.name = "Date"

            train = df[df.index < self.split_date]
            test = df[df.index >= self.split_date]

            if len(train) < 20 or len(test) < 5:
                continue

            ma = (
                df["Close"]
                .rolling(self.ma_window)
                .mean()
                .loc[test.index]
            )

            ema = (
                df["Close"]
                .ewm(span=self.ma_window, adjust=False)
                .mean()
                .loc[test.index]
            )

            ar_model = AutoReg(train["Close"], lags=self.ar_lags).fit()
            ar_pred = ar_model.predict(
                start=len(train),
                end=len(train) + len(test) - 1
            ).values

            arima_model = ARIMA(
                train["Close"], order=self.arima_order
            ).fit()
            arima_pred = arima_model.forecast(len(test)).values

            gbm_pred = self.gbm(train["Close"], test["Close"]).values

            preds = {
                "MA": ma.values,
                "EMA": ema.values,
                "AR": ar_pred,
                "ARIMA": arima_pred,
                "GBM": gbm_pred
            }

            for model, yhat in preds.items():
                records.append({
                    "Ticker": ticker,
                    "Model": model,
                    "RMSE": self.rmse(test["Close"].values, yhat),
                    "SMAPE": self.smape(test["Close"].values, yhat)
                })

        df_metrics = pd.DataFrame(records)

        out = (
            df_metrics
            .set_index(["Ticker", "Model"])
            .unstack("Model")
            .swaplevel(0, 1, axis=1)
            .sort_index(axis=1)
        )

        return out


In [8]:
tickers = [
    "RELIANCE.NS",     # Large-cap
    "TCS.NS",          # Large-cap
    "HDFCBANK.NS",     # Large-cap
    "PIDILITIND.NS",   # Mid-cap
    "AUBANK.NS",       # Mid-cap
    "IEX.NS",          # Small-cap
    "MAPMYINDIA.NS",   # Small-cap
    "LTTS.NS",          # Small-cap / mid-tech
    "AAPL",     # Large-cap
    "MSFT",     # Large-cap
    "GOOGL",    # Large-cap
    "AMZN",     # Large-cap
    "NVDA",     # Large-cap
    "CRWD",     # Mid-cap
    "PLTR",     # Mid-cap
    "FSLR",     # Mid-cap
    "CELH",     # Small-cap
    "U",         # Small-cap / mid-tech
    "HSBA.L",   # Large-cap
    "BP.L",     # Large-cap
    "ULVR.L",   # Large-cap
    "AUTO.L",   # Mid-cap
    "JDW.L",    # Mid-cap
    "ASC.L",    # Small-cap
]

In [11]:
!pip install xlsxwriter
exp = StockExperiment(
    tickers=tickers,
    start='2018-03-01',
    end='2025-04-30',
    split_date='2025-04-01'

)
df = exp.run()

# The df is already pivoted with 'Ticker' as index and a MultiIndex for columns (Model, Metric).
# The error "KeyError: 'Ticker'" occurred because 'Ticker' is already an index, not a column.
# Also, 'MAE' and 'R2' are not generated by the StockExperiment class.
# We will directly use 'df' and reformat its columns to match the styling intent.

df_pivoted = df

# To align with the intended reindex (columns=['MAE', 'RMSE', 'R2'], level=0),
# which suggests grouping by metric, we need to swap the column levels.
# Current columns: (Model, Metric) -> e.g., ('AR', 'RMSE')
# Desired columns for reindex: (Metric, Model) -> e.g., ('RMSE', 'AR')
df_pivoted = df_pivoted.swaplevel(0, 1, axis=1)

# Sort the columns to ensure consistent ordering, e.g., RMSE metrics grouped together
df_pivoted = df_pivoted.sort_index(axis=1)

# Reindex to select only the available metrics ('RMSE', 'SMAPE') at level 0.
# The original code asked for 'MAE', 'RMSE', 'R2'. We can only provide 'RMSE' and 'SMAPE'.
df_pivoted = df_pivoted.reindex(columns=['RMSE', 'SMAPE'], level=0)

styled_df = df_pivoted.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f7f7f7')]},
    {'selector': 'td', 'props': [('text-align', 'center')]} # Optional: centers the data values too
])

styled_df.to_excel("Stock_Metrics_basic.xlsx", engine='xlsxwriter')
styled_df

/tmp/ipython-input-2056240380.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/deterministic.py:308: UserWarning: Only PeriodIndexes, Datet